In [1]:
#
# %pip install dask
%pip freeze

anyio @ file:///C:/ci_311/anyio_1676425491996/work/dist
appdirs==1.4.4
archspec @ file:///croot/archspec_1697725767277/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci_311/argon2-cffi-bindings_1676424443321/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/async-lru_1699554572212/work
attrs @ file:///C:/b/abs_35n0jusce8/croot/attrs_1695717880170/work
Babel @ file:///C:/ci_311/babel_1676427169844/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///C:/b/abs_0agyz1wsr4/croot/beautifulsoup4-split_1681493048687/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
boltons @ file:///C:/ci_311/boltons_1677729932371/work
Brotli @ file:///C:/ci_311/brotli-split_1676435766766/work
bs4==0.0.1
certifi @ file:///C:/b/abs_91u83siphd/croot/certifi_1700501720658/work/certifi
cffi @ file:///C:/b/abs_924gv1kxzj/c

In [2]:


import config as cfg
import datetime
from itertools import product
import json
import logging
import os
import pandas as pd
import sklearn_util as sklu

import pickle
import trader as trader
import importlib
importlib.reload(cfg)
importlib.reload(trader)
importlib.reload(sklu)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

if not os.path.exists('logs'):
    os.makedirs('logs')
filename = f"logs/{__name__}_{datetime.datetime.now().strftime('%Y-%m-%d-%H')}.log"
if not os.path.exists(filename):
    with open(filename, 'wt') as f:
        f.write('Starting log file: ' + filename + '\n')
        
if len(logger.handlers) < 1:
    handler = logging.FileHandler(filename)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

force_rebuild = True

config_path = "config"
# config_file = "debug.json"
# config_file = "debug_dual_forecast_out.json"
# config_file = "debug_dual_forecast_out_6_months.json"
# config_file = "backtest_2022.json"
# config_file = "backtest_2023.json"
# config_file = "daily_run_backtest_2023.json"
config_file = "daily_run.json"
try:
    conf = cfg.load_config(config_path, config_file)
except json.JSONDecodeError as e:
    logger.error(f"Error loading JSON file: {config_file}. {e}")

#     # 'log_regression',
#     # 'RFClassifier',
#     # 'MLPClassifier',
model_type = 'log_regression'
date_col = conf.date_col
forecast_actuals_df = None
pickle_file_name = os.path.join(conf.results_path, f"forecast_actuals_all_models_class.pkl")
if not force_rebuild and os.path.exists(pickle_file_name):
    logger.info(f"Loading pickle file: {pickle_file_name}")
    with open(pickle_file_name, 'rb') as file:
        forecast_actuals_df = pickle.load(file)
else:
    forecast_actuals_list = []
    for training_config in conf.training_configs:
        df = cfg.load_results_pickle_file_class(conf, training_config, model_type)
        df = df.drop_duplicates()
        df['model_type'] = model_type
        # display(df)
        forecast_actuals_list.append(df)
    logger.info(f"Saving pickle file: {pickle_file_name}")
    logger.info(f"forecast_actuals_list: {forecast_actuals_list}")
    forecast_actuals_df = pd.concat(forecast_actuals_list, axis=0)
    with open(pickle_file_name, 'wb') as file:
        pickle.dump(forecast_actuals_df, file)
forecast_actuals_df = forecast_actuals_df.sort_index()
forecast_actuals_df


2024-02-20 21:23:20,306 - config - INFO - loading config at path config\daily_run.json
2024-02-20 21:23:20,363 - config - INFO - Loading pickle file: results\2024-02-20\log_regression_LogadjClose_3_rolling_mean_10000_smpl_7_day_0_75_confUp_0_7_confDown_0_003_changeUp_-0_003_changeDown.pkl
2024-02-20 21:23:20,464 - config - INFO - Loading pickle file: results\2024-02-20\log_regression_LogadjClose_7_rolling_mean_10000_smpl_14_day_0_75_confUp_0_7_confDown_0_003_changeUp_-0_003_changeDown.pkl
2024-02-20 21:23:20,522 - config - INFO - Loading pickle file: results\2024-02-20\log_regression_LogadjClose_7_rolling_mean_10000_smpl_21_day_0_75_confUp_0_7_confDown_0_003_changeUp_-0_003_changeDown.pkl
2024-02-20 21:23:20,570 - __main__ - INFO - Saving pickle file: results\2024-02-20\forecast_actuals_all_models_class.pkl
2024-02-20 21:23:20,606 - __main__ - INFO - forecast_actuals_list: [                   index   close  adjClose     high  LogadjClose  \
symbol date                                  

index   close  adjClose    high  LogadjClose  \
symbol date                                                       
A      2024-01-02      0  138.75    138.75  140.59     4.932674   
       2024-01-02      0  138.75    138.75  140.59     4.932674   
       2024-01-02      0  138.75    138.75  140.59     4.932674   
       2024-01-03      1  131.16    131.16  138.00     4.876418   
       2024-01-03      1  131.16    131.16  138.00     4.876418   
...                  ...     ...       ...     ...          ...   
ZTS    2024-03-05  22709     NaN       NaN     NaN          NaN   
       2024-03-06  22710     NaN       NaN     NaN          NaN   
       2024-03-07  22711     NaN       NaN     NaN          NaN   
       2024-03-08  22712     NaN       NaN     NaN          NaN   
       2024-03-12  22713     NaN       NaN     NaN          NaN   

                   LogadjClose_3_rolling_mean label_date    open      low  \
symbol date                                                                 
A      2024-01-02                    4.936612 2024-01-09  138.19  137.910   
       2024-01-02                         NaN 2024-01-16  138.19  137.910   
       2024-01-02                         NaN 2024-01-23  138.19  137.910   
       2024-01-03                    4.938083 2024-01-10  138.00  131.065   
       2024-01-03                         NaN 2024-01-17  138.00  131.065   
...                                       ...        ...     ...      ...   
ZTS    2024-03-05                         NaN        NaT     NaN      NaN   
       2024-03-06                         NaN        NaT     NaN      NaN   
       2024-03-07                         NaN        NaT     NaN      NaN   
       2024-03-08                         NaN        NaT     NaN      NaN   
       2024-03-12                         NaN        NaT     NaN      NaN   

                  label_date_reverse  ...  forecast_leading  forecast_target  \
symbol date                           ...                                      
A      2024-01-02                NaT  ...               0.0              NaN   
       2024-01-02                NaT  ...               0.0              NaN   
       2024-01-02                NaT  ...               0.0              NaN   
       2024-01-03                NaT  ...              -1.0              NaN   
       2024-01-03                NaT  ...              -1.0              NaN   
...                              ...  ...               ...              ...   
ZTS    2024-03-05         2024-02-13  ...               NaN              0.0   
       2024-03-06         2024-02-14  ...               NaN              0.0   
       2024-03-07         2024-02-15  ...               NaN              0.0   
       2024-03-08         2024-02-16  ...               NaN              0.0   
       2024-03-12         2024-02-20  ...               NaN              0.0   

                           actual_column_name  forecast_leading_days actual  \
symbol date                                                                   
A      2024-01-02  LogadjClose_3_rolling_mean                      7   None   
       2024-01-02  LogadjClose_7_rolling_mean                     14   None   
       2024-01-02  LogadjClose_7_rolling_mean                     21   None   
       2024-01-03  LogadjClose_3_rolling_mean                      7   None   
       2024-01-03  LogadjClose_7_rolling_mean                     14   None   
...                                       ...                    ...    ...   
ZTS    2024-03-05  LogadjClose_7_rolling_mean                     21   None   
       2024-03-06  LogadjClose_7_rolling_mean                     21   None   
       2024-03-07  LogadjClose_7_rolling_mean                     21   None   
       2024-03-08  LogadjClose_7_rolling_mean                     21   None   
       2024-03-12  LogadjClose_7_rolling_mean                     21   None   

                   conf_thrs     chng_thrs      model_type  \
symbol date   

In [3]:

import trader as trader
import importlib
importlib.reload(trader)

# data_handler_joined = None
# file_path = pickle_file_name # 'results/forecast_actuals_all_models_class.pkl'
# for training_config in conf.training_configs:  

grouping_dimensions = ['symbol', 'model_type']
forecast_dimensions = ['adjClose', 'low', 'open', 'high']
forecast_leading_col = 'forecast_leading'
forecast_target_col = 'forecast_target'
actual_col_name = 'actual_column_name'
forecast_leading_days_col = 'forecast_leading_days'
class_actual_col = 'class_actual'
attribute_column_names=[  
        'conf_thrs',
        'chng_thrs',
    ]
pivot_columns=['forecast_target', 'forecast_leading']
# forecast_leading_days = [1, 3, 5, 10, 15]
# forecast_class_thresholds = [0.6]
data_handler = trader.DataHandler(
    grouping_dimensions=grouping_dimensions,
    forecast_dimensions=forecast_dimensions,
    date_col=date_col,
    forecast_leading_col=forecast_leading_col,
    forecast_target_col=forecast_target_col,
    actual_col_name=actual_col_name,
    forecast_leading_days_col = forecast_leading_days_col,
    class_actual_col=class_actual_col,
    attribute_column_names=attribute_column_names,
    # file_path=os.path.join(conf.results_path, training_config.create_filename(model_type) + ".pkl"),
    file_path=pickle_file_name
)


In [4]:
forecast_actuals_df.index.get_level_values('date').max()

Timestamp('2024-03-12 00:00:00')

In [34]:


import trader
import importlib
importlib.reload(trader)

initial_cash = 10000
portfolio = trader.Portfolio(initial_cash=initial_cash) # initialize before functions declared

def avg_close_up_or_down(symbol, symbol_data):
    # logger.info(f"avg_close_up_or_down: symbol: {symbol} symbol_data: {symbol_data}")
    symbol_data = symbol_data.iloc[0]
    direction_prediction = symbol_data[f'forecast_leading_{conf.forecast_col}_{conf.forecast_leading_day}_{conf.forecast_threshold}']
    buy = 5 # dollars
    sell = 10 # dollars
    if direction_prediction is None:
        return []
    elif direction_prediction > 0:
        return [trader.Order(symbol=symbol, operation='buy', quantity=(buy / symbol_data['adjClose']), price=symbol_data['adjClose'])]
    elif direction_prediction < 0:
        return [trader.Order(symbol=symbol, operation='sell', quantity=(sell / symbol_data['adjClose']), price=symbol_data['adjClose'])]
    return []

def just_buy_everything_equally(symbol, symbol_data):
    symbol_data = symbol_data.iloc[0]
    bet = 5
    return [trader.Order(symbol=symbol, operation='buy', quantity=(bet / symbol_data['adjClose']), price=symbol_data['adjClose'])]

def avg_close_up_or_down_keep_cash_close_mid_mid(symbol, symbol_data):
    # logger.info(f"avg_close_up_or_down: symbol: {symbol} symbol_data: {symbsol_data}")
    symbol_data = symbol_data.iloc[0]
    direction_prediction = symbol_data[f'forecast_leading_{conf.forecast_col}_{conf.forecast_leading_day}_{conf.forecast_threshold}']
    desired = 3000
    cash = portfolio.get_cash()
    bet = 5
    sell = max(bet * ((desired - cash) / cash), 1)
    buy = max(bet * ((cash - desired) / desired), 1)
    close = symbol_data['adjClose']
    mid = (symbol_data['high'] + symbol_data['low']) / 2
    
    orders = [] 
    
    # order smoothed to middle for safety
    buy_price = (close + mid) / 2 # min(close, mid)
    sell_price = (close + mid) / 2
    
    if direction_prediction > 0:
        orders.append(trader.Order(symbol=symbol, operation='buy', quantity=(buy / buy_price), price=buy_price))
    elif direction_prediction < 0:     
        orders.append(trader.Order(symbol=symbol, operation='sell', quantity=(sell / sell_price), price=sell_price))

        
    return orders

def avg_close_up_or_down_keep_cash_min_max(symbol, symbol_data):
    # logger.info(f"avg_close_up_or_down: symbol: {symbol} symbol_data: {symbsol_data}")
    symbol_data = symbol_data.iloc[0]
    direction_prediction = symbol_data[f'forecast_leading_{conf.forecast_col}_{conf.forecast_leading_day}_{conf.forecast_threshold}']
    desired = 5000
    cash = portfolio.get_cash()
    bet = 1
    sell = max(bet * ((desired - cash) / cash), 1)
    buy = max(bet * ((cash - desired) / desired), 1)
    close = symbol_data['adjClose']
    mid = (symbol_data['high'] + symbol_data['low']) / 2
    
    orders = [] 
    
    # min/max as goal
    buy_price = min(close, mid)
    sell_price = max(close, mid)
    
    if direction_prediction > 0:
        orders.append(trader.Order(symbol=symbol, operation='buy', quantity=(buy / buy_price), price=buy_price))
    elif direction_prediction < 0:     
        orders.append(trader.Order(symbol=symbol, operation='sell', quantity=(sell / sell_price), price=sell_price))
        
    return orders


def avg_close_up_or_down_keep_cash_min_max_plus_close_mid_mid(data_by_date):
    
    import numpy as np
    data = data_by_date.copy(deep=True)
    data['net_direction_prediction'] = 0
    for forecast_leading_col in [col for col in data_by_date.columns if 'forecast_leading' in col]:
        data['net_direction_prediction'] += data[forecast_leading_col]
    data['abs_net_direction_prediction'] = np.abs(data['net_direction_prediction'])    
    data['quantity'] = 1.0
    scale_by_price = True
    if scale_by_price:
        bet = 1
        data['quantity'] *= (bet / data['adjClose'])
    scale_by_cash = True
    sell_scalar = 1.0
    buy_scalar = 1.0
    if scale_by_cash:
        desired = 8000
        buffer = 1000
        cash = portfolio.get_cash()
        # data['quantity'] *= max((desired - cash) / max(cash, 0.001), 1)
        if (desired - cash) > buffer:
            sell_scalar = 1+((abs(desired - cash)) / max(cash, 1))
        # elif (cash - desired) > buffer:
        #     buy_scalar = 1+((abs(cash - desired)) / max(desired, 1))
        
    # can't buy less than a dollar worth
    data['quantity'] = np.maximum((1 / data['adjClose']), data['quantity'])
        
    close = data['adjClose']
    data['mid'] = (data['high'] + data['low']) / 2
    
    data['quantity'] *= data['abs_net_direction_prediction']
        
    # order smoothed to increase likelihood of execution
    data['buy_price_lazy'] = (data['adjClose'] + data['mid']) / 2 # min(close, mid)
    data['sell_price_lazy'] = (data['adjClose'] + data['mid']) / 2
    
    # min/max as goal for aggressive bids (might not go through because of big traders snapping them up at good prices)
    data['buy_price_aggressive'] = np.minimum(data['adjClose'], data['mid'])
    data['sell_price_aggressive'] = np.maximum(data['adjClose'], data['mid'])
        
    orders = []
    for row in data.reset_index().sort_values(by='abs_net_direction_prediction', ascending=False).iterrows():
        row = row[1]
        quantity = row.quantity
        symbol = row.symbol
        if row.net_direction_prediction > 0:
            orders.append(trader.Order(symbol=symbol, operation='buy', quantity=quantity*buy_scalar, price=row.buy_price_lazy))
            orders.append(trader.Order(symbol=symbol, operation='buy', quantity=quantity*buy_scalar, price=row.buy_price_aggressive))
        elif row.net_direction_prediction < 0:     
            orders.append(trader.Order(symbol=symbol, operation='sell', quantity=quantity*sell_scalar, price=row.sell_price_lazy))
            orders.append(trader.Order(symbol=symbol, operation='sell', quantity=quantity*sell_scalar, price=row.sell_price_aggressive))
        # elif cash < desired:
        #     orders.append(trader.Order(symbol=symbol, operation='sell', quantity=quantity, price=row.sell_price_aggressive))
        #     orders.append(trader.Order(symbol=symbol, operation='sell', quantity=quantity, price=row.sell_price_lazy))
             
    return orders


data_start_raw = data_handler.dataframe.index.get_level_values(date_col).min()
data_start_pivoted = data_handler.pivoted_forecast_dimensions_df.index.get_level_values(date_col).min()
start_date = conf.end_date + datetime.timedelta(days=-(conf.total_holdout_base)) # need the previous holdout days for actual copmarisons
end_date = conf.end_date
logger.info(f"**** RUN START TIME: {datetime.datetime.now()} start_date:{start_date} conf.start_date: {conf.start_date} conf.end_date: {conf.end_date}" +
            f" conf.largest_forecast_out: {conf.largest_forecast_out} data_start_raw: {data_start_raw} data_start_pivoted:{data_start_pivoted}  initial_cash: {initial_cash}" +
            f" model_type: {model_type}")
for training_config in conf.training_configs:
    logger.info(
    f" - forecast_col:{training_config.forecast_col} forecast_out: {training_config.forecast_out} forecast_mean_trailing_days: {training_config.forecast_mean_trailing_days} " +
    f" classify_confidence_threshold_up: {training_config.classify_confidence_threshold_up} classify_confidence_threshold_down: {training_config.classify_confidence_threshold_down} " +
    f" classify_change_threshold_ratio_up: {training_config.classify_change_threshold_ratio_up} classify_change_threshold_ratio_down: {training_config.classify_change_threshold_ratio_down} "
    )

with open ('simulation_output.txt', 'wt') as file:
    file.write('')
    
    trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
    strategy_simulator = trader.StrategySimulator(
        model_type=model_type, 
        data_handler=data_handler, 
        trade_simulator=trade_simulator,
        strategy=avg_close_up_or_down_keep_cash_min_max_plus_close_mid_mid, 
        ) #  log_regression
    strategy_simulator.simulate_strategy(
        #  datetime.datetime(2021,6,1), 
        start_date,
        end_date,
        expiration_duration=1)

logger.info(f"end time: {datetime.datetime.now()}")
# transaction_log

import logging
import trader as trader
import timing
import importlib
importlib.reload(trader)
importlib.reload(timing)

transaction_log_df = trade_simulator.portfolio.transaction_log

logs_path = "logs"
if not os.path.exists(logs_path):
    os.makedirs(logs_path)
filename = f"transaction_log_{datetime.datetime.now().strftime('%Y-%m-%d')}.log"
filename = os.path.join(logs_path, filename)
saved = transaction_log_df.dataframe[pd.to_datetime(transaction_log_df.dataframe['execution_date']) >= datetime.datetime.now() - datetime.timedelta(days=2)]
saved.to_csv(filename, mode = 'a')

logger = logging.getLogger(__name__)
handler = logging.FileHandler(filename)
logger.addHandler(handler)

strategy_simulator.print_balance(initial_cash, 
                                 start_date, 
                                 end_date, 
                                 in_logger=logger)

# for order, ts in strategy_simulator.trade_simulator.get_orders():
#     logger.info(f"order: {order.to_string()} ts: {ts}")

# 2024-02-19 15:05:33,130 - __main__ - INFO - **** RUN END TIME: 2024-02-19 15:05:33.130673 start_date:2023-01-05 00:00:00 
# conf.start_date: 2020-01-01 00:00:00 conf.end_date: 2023-12-31 00:00:00 conf.largest_forecast_out: 7 
# data_start_raw: 2022-12-22 00:00:00 data_start_pivoted:2022-12-22 00:00:00  initial_cash: 10000 model_type: log_regression 
# model_configs: ['7_days_out_3_MA_today_30_holdout.json'] 

logger.info(f"**** RUN END TIME: {datetime.datetime.now()} start_date:{start_date} conf.start_date: {conf.start_date} conf.end_date: {conf.end_date}\n" +
            f" conf.largest_forecast_out: {conf.largest_forecast_out} data_start_raw: {data_start_raw} data_start_pivoted:{data_start_pivoted}  initial_cash: {initial_cash}" +
            f" model_type: \n {model_type} model_configs: {conf.model_configs} ")
for training_config in conf.training_configs:
    logger.info(
    f" - forecast_col:{training_config.forecast_col} forecast_out: {training_config.forecast_out} forecast_mean_trailing_days: {training_config.forecast_mean_trailing_days}" +
    f" day_offsets: {conf.day_offsets} total_holdout_base:{conf.total_holdout_base} \n" +
    f" classify_confidence_threshold_up: {training_config.classify_confidence_threshold_up} classify_confidence_threshold_down: {training_config.classify_confidence_threshold_down} " +
    f" classify_change_threshold_ratio_up: {training_config.classify_change_threshold_ratio_up} classify_change_threshold_ratio_down: {training_config.classify_change_threshold_ratio_down} "
    )

logger.removeHandler(handler)
handler.close()


2024-02-20 21:50:34,394 - __main__ - INFO - **** RUN START TIME: 2024-02-20 21:50:34.394655 start_date:2024-01-21 21:23:20.362684 conf.start_date: 2021-01-01 00:00:00 conf.end_date: 2024-02-20 21:23:20.362684 conf.largest_forecast_out: 21 data_start_raw: 2023-12-31 00:00:00 data_start_pivoted:2023-12-31 00:00:00  initial_cash: 10000 model_type: log_regression
2024-02-20 21:50:34,395 - __main__ - INFO -  - forecast_col:LogadjClose_3_rolling_mean forecast_out: 7 forecast_mean_trailing_days: 3  classify_confidence_threshold_up: 0.75 classify_confidence_threshold_down: 0.7  classify_change_threshold_ratio_up: 0.003 classify_change_threshold_ratio_down: -0.003 
2024-02-20 21:50:34,398 - __main__ - INFO -  - forecast_col:LogadjClose_7_rolling_mean forecast_out: 14 forecast_mean_trailing_days: 7  classify_confidence_threshold_up: 0.75 classify_confidence_threshold_down: 0.7  classify_change_threshold_ratio_up: 0.003 classify_change_threshold_ratio_down: -0.003 
2024-02-20 21:50:34,400 - __mai

In [35]:

for order, ts in strategy_simulator.trade_simulator.get_orders():
    logger.info(f"order: {order.to_string()} ts: {ts}")

2024-02-20 21:51:30,159 - __main__ - INFO - order: Order(symbol=PEAK, operation=sell, quantity=0.5585073925105662, price=16.9525) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,162 - __main__ - INFO - order: Order(symbol=PEAK, operation=sell, quantity=0.5585073925105662, price=16.985) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,165 - __main__ - INFO - order: Order(symbol=TPR, operation=buy, quantity=0.10692899914456801, price=46.5025) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,166 - __main__ - INFO - order: Order(symbol=TPR, operation=buy, quantity=0.10692899914456801, price=46.245000000000005) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,167 - __main__ - INFO - order: Order(symbol=RL, operation=buy, quantity=0.02727024815925825, price=182.3875) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,168 - __main__ - INFO - order: Order(symbol=RL, operation=buy, quantity=0.02727024815925825, price=181.425) ts: 2024-02-20 00:00:00
2024-02-20 21:51:30,171 - __main__ - INFO - order: Order(symbol=CZR, op

In [36]:
# transaction_log_df.dataframe
trades_path = "trades" 
if not os.path.exists(trades_path):
    os.makedirs(trades_path)
filename = f"trades_{datetime.datetime.now().strftime('%Y-%m-%d-%H-%m-%S')}"
filename = os.path.join(trades_path, filename + ".csv")
fresh_trades = transaction_log_df.dataframe[pd.to_datetime(transaction_log_df.dataframe['execution_date']) >= (datetime.datetime.now() - datetime.timedelta(days=2))]
fresh_trades.to_csv(filename)
fresh_trades

,symbol,operation,quantity,price,execution_date


In [37]:
portfolio.get_holdings()

{'NVDA': (0.1874329846437965, 730.87),
 'ANET': (0.23752976917878899, 263.34000000000003),
 'TSCO': (0.15608163335896372, 232.6225),
 'TRV': (0.2064698835689931, 216.62),
 'BLDR': (0.3225151743353156, 185.29000000000002),
 'AZO': (0.017980830353885396, 2724.1800000000003),
 'LRCX': (0.07619844681820029, 915.7),
 'CRM': (0.16876311049746567, 287.04999999999995),
 'MAR': (0.206959971755319, 247.57),
 'META': (0.20805407521443917, 470.10005),
 'DLR': (0.26455214684255285, 140.825),
 'KLAC': (0.07000914057455329, 652.39),
 'ORLY': (0.054041219677145785, 1042.615),
 'AJG': (0.10067092063321578, 234.80345),
 'ALL': (0.3188273731227842, 159.855),
 'TDG': (0.07023722517736754, 1138.0525),
 'AMD': (0.31222853257133953, 175.81),
 'GWW': (0.07825292049395773, 943.495),
 'IBM': (0.32118735546711946, 183.57),
 'IT': (0.05598384795206437, 450.97),
 'NOW': (0.059474350891758496, 784.285),
 'AVGO': (0.04357409759172883, 1257.15895),
 'ISRG': (0.17878520284130076, 379.95500000000004),
 'FICO': (0.03305

In [38]:
data_by_date = data_handler.get_stock_data_normalized()
# data_by_date = data_by_date[data_by_date.index.get_level_values(data_handler.date_col) == pd.to_datetime(2024, 2, 1)]
data_by_date = data_by_date[data_by_date.index.get_level_values('date')  == datetime.datetime(2024, 2, 1)]
data_by_date

,,,adjClose,low,open,high,forecast_target_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_target_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_target_LogadjClose_7_rolling_mean_21_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_7_rolling_mean_21_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs
symbol,model_type,date,,,,,,,,,,
A,log_regression,2024-02-01,133.05,129.2605,130.06,133.0500,0.0,-1.0,-1.0,0.0,0.0,0.0
AAL,log_regression,2024-02-01,14.34,13.8727,14.35,14.3999,1.0,-1.0,0.0,-1.0,0.0,0.0
AAPL,log_regression,2024-02-01,186.62,183.8200,183.99,186.9500,0.0,0.0,0.0,0.0,0.0,0.0
ABBV,log_regression,2024-02-01,167.59,162.6700,163.70,167.6800,0.0,0.0,0.0,0.0,0.0,0.0
ABNB,log_regression,2024-02-01,146.49,143.6500,144.80,146.8500,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,log_regression,2024-02-01,130.45,128.0150,129.31,130.4800,0.0,0.0,0.0,0.0,0.0,0.0
ZBH,log_regression,2024-02-01,127.25,124.1350,125.43,127.2500,0.0,0.0,0.0,1.0,1.0,0.0
ZBRA,log_regression,2024-02-01,242.60,233.9500,239.55,242.8700,1.0,-1.0,0.0,-1.0,-1.0,-1.0


In [39]:
data_handler.get_stock_data_raw()


close  adjClose    high  LogadjClose  \
symbol model_type     date                                                
A      log_regression 2024-01-02  138.75    138.75  140.59     4.932674   
                      2024-01-02  138.75    138.75  140.59     4.932674   
                      2024-01-02  138.75    138.75  140.59     4.932674   
                      2024-01-03  131.16    131.16  138.00     4.876418   
                      2024-01-03  131.16    131.16  138.00     4.876418   
...                                  ...       ...     ...          ...   
ZTS    log_regression 2024-03-05     NaN       NaN     NaN          NaN   
                      2024-03-06     NaN       NaN     NaN          NaN   
                      2024-03-07     NaN       NaN     NaN          NaN   
                      2024-03-08     NaN       NaN     NaN          NaN   
                      2024-03-12     NaN       NaN     NaN          NaN   

                                  LogadjClose_3_rolling_mean label_date  \
symbol model_type     date                                                
A      log_regression 2024-01-02                    4.936612 2024-01-09   
                      2024-01-02                         NaN 2024-01-16   
                      2024-01-02                         NaN 2024-01-23   
                      2024-01-03                    4.938083 2024-01-10   
                      2024-01-03                         NaN 2024-01-17   
...                                                      ...        ...   
ZTS    log_regression 2024-03-05                         NaN        NaT   
                      2024-03-06                         NaN        NaT   
                      2024-03-07                         NaN        NaT   
                      2024-03-08                         NaN        NaT   
                      2024-03-12                         NaN        NaT   

                                    open      low label_date_reverse  \
symbol model_type     date                                             
A      log_regression 2024-01-02  138.19  137.910                NaT   
                      2024-01-02  138.19  137.910                NaT   
                      2024-01-02  138.19  137.910                NaT   
                      2024-01-03  138.00  131.065                NaT   
                      2024-01-03  138.00  131.065                NaT   
...                                  ...      ...                ...   
ZTS    log_regression 2024-03-05     NaN      NaN         2024-02-13   
                      2024-03-06     NaN      NaN         2024-02-14   
                      2024-03-07     NaN      NaN         2024-02-15   
                      2024-03-08     NaN      NaN         2024-02-16   
                      2024-03-12     NaN      NaN         2024-02-20   

                                  LogadjClose_3_rolling_mean_reverse  \
symbol model_type     date                                             
A      log_regression 2024-01-02                                 NaN   
                      2024-01-02                                 NaN   
                      2024-01-02                                 NaN   
                      2024-01-03                                 NaN   
                      2024-01-03                                 NaN   
...                                                              ...   
ZTS    log_regression 2024-03-05                                 NaN   
                      2024-03-06                                 NaN   
                      2024-03-07                                 NaN   
                      2024-03-08                                 NaN   
                      2024-03-12                                 NaN   

                                  class_actual  forecast_leading  \
symbol model_type     date                                         
A      log_regression 2024-01-02             0               0.0   
     

In [40]:
data_handler.pivoted_forecast_dimensions_df.dropna()

adjClose      low    open     high  \
symbol model_type     date                                             
A      log_regression 2024-01-02    138.75  137.910  138.19  140.590   
                      2024-01-03    131.16  131.065  138.00  138.000   
                      2024-01-04    131.00  130.190  130.55  131.495   
                      2024-01-05    130.56  128.620  130.00  131.960   
                      2024-01-08    133.38  129.810  130.14  133.570   
...                                    ...      ...     ...      ...   
ZTS    log_regression 2024-02-13    183.49  182.070  186.60  190.610   
                      2024-02-14    184.08  182.100  185.19  186.490   
                      2024-02-15    189.65  183.860  183.86  190.340   
                      2024-02-16    188.39  187.930  189.40  190.350   
                      2024-02-20    186.55  186.240  187.30  189.410   

                                  forecast_target_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs  \
symbol model_type     date                                                                                                                       
A      log_regression 2024-01-02                                                0.0                                                              
                      2024-01-03                                                0.0                                                              
                      2024-01-04                                                0.0                                                              
                      2024-01-05                                                0.0                                                              
                      2024-01-08                                                0.0                                                              
...                                                                             ...                                                              
ZTS    log_regression 2024-02-13                                                1.0                                                              
                      2024-02-14                                                1.0                                                              
                      2024-02-15                                                0.0                                                              
                      2024-02-16                                                0.0                                                              
                      2024-02-20                                               -1.0                                                              

                                  forecast_target_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs  \
symbol model_type     date                                                                                                                        
A      log_regression 2024-01-02                                                0.0                                                               
                      2024-01-03                                                0.0                                                               
                      2024-01-04                                                0.0                                                               
                      2024-01-05                                                0.0                                                               
                      2024-01-08                                                0.0                                                               
...                                                                             ...                                                               
ZTS    log_regression 

In [41]:
data_handler.pivoted_forecast_dimensions_df[data_handler.pivoted_forecast_dimensions_df.index.get_level_values('symbol') == 'MSFT'].sum()

adjClose                                                                                                          13851.200
low                                                                                                               13756.680
open                                                                                                              13859.145
high                                                                                                              13973.181
forecast_target_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs          0.000
forecast_target_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs        10.000
forecast_target_LogadjClose_7_rolling_mean_21_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs         5.000
forecast_leading_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs         0.000
forecast

In [42]:
multi_count_df = data_handler.pivoted_forecast_dimensions_df.reset_index().groupby(data_handler.grouping_dimensions + [date_col])[date_col].count()
multi_count_df = multi_count_df[multi_count_df > 1]
multi_count_df


Series([], Name: date, dtype: int64)

In [43]:
data_handler.pivoted_forecast_dimensions_df[data_handler.pivoted_forecast_dimensions_df.index.get_level_values(date_col) == '2023-01-31'] 


,,,adjClose,low,open,high,forecast_target_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_target_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_target_LogadjClose_7_rolling_mean_21_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_3_rolling_mean_7_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_7_rolling_mean_14_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs,forecast_leading_LogadjClose_7_rolling_mean_21_forecast_leading_days_0_75_0_7_conf_thrs_0_003_-0_003_chng_thrs
symbol,model_type,date,,,,,,,,,,


In [44]:
data_handler.pivoted_forecast_dimensions_df.dropna().index.get_level_values(date_col).max()

Timestamp('2024-02-20 00:00:00')

In [45]:
# import sklearn_util as sklu
# import importlib
# importlib.reload(sklu)

# all_metrics = []
# for model_type in model_types:
#     df = forecast_actuals_df[forecast_actuals_df['model_type'] == model_type]
#     metrics, metrics_str = sklu.calculate_metrics(df, 'forecast_target', 'class_actual')
#     logger.info(f"model_type:{model_type} \n {metrics_str}")
#     all_metrics.append(metrics)
    
# metrics, metrics_str = sklu.calculate_metrics(forecast_actuals_df, 'forecast_target', 'High')
# logger.info(metrics_str)
# metrics

In [46]:
# metrics, metrics_str = sklu.calculate_metrics(forecast_actuals_df, 'forecast_target', 'Low')
# logger.info(metrics_str)
# metrics

In [47]:
# import trader as trader
# import importlib
# importlib.reload(trader)

# grouping_dimensions = ['symbol', 'model_type']
# forecast_dimensions = ['High', 'Low']
# date_col = 'date'
# forecast_leading_col = 'forecast_leading'
# forecast_target_col = 'forecast_target'
# actual_col_name = 'actual_column_name'
# forecast_leading_days_col = 'forecast_leading_days'
# forecast_leading_days = [1, 2, 3, 4]
# file_path = None
# df = pd.DataFrame({
#     'symbol': ['AAPL', 'AAPL', 'GOOG', 'GOOG'],
#     'model_type': ['type1', 'type2', 'type1', 'type2'],
#     'forecast': [100, 200, 300, 400],
#     'date': pd.to_datetime(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04']),
#     'forecast_leading': [1, 2, 3, 4],
#     'forecast_target': [100, 200, 300, 400],
#     'High': [90, 210, 290, 410],
#     'Low': [80, 125, 145, 365],
#     'actual_column_name': ['High', 'High', 'Low', 'Low'],
#     'forecast_leading_days': [1, 2, 3, 4]
# })
# test_df = pd.DataFrame({
#     'symbol': ['AAPL', 'AAPL'],
#     'model_type': ['type1', 'type2'],
#     'forecast': [100, 200],
#     'date': pd.to_datetime(['2022-01-01', '2022-01-02']),
#     'forecast_leading': [1, 2],
#     'forecast_target': [100, 200],
#     'High': [90, 210],
#     'Low': [80, 125],
#     'actual_column_name': ['High', 'High'],
#     'forecast_leading_days': [1, 2]
# })
# data_handler = trader.DataHandler(
#     grouping_dimensions, 
#     forecast_dimensions, 
#     date_col,
#     forecast_leading_col, 
#     forecast_target_col,
#     actual_col_name,
#     forecast_leading_days_col,
#     forecast_leading_days,
#     file_path, 
#     df
# )
# data_handler.dataframe

In [48]:
# def pivot_forecasts(df_input):
    
#     df = df_input.copy(deep=True)

#     # Assuming df is your DataFrame
#     # Replace 'your_data.csv' with your actual data file
#     # df = pd.read_csv('your_data.csv')

#     # Creating a unique identifier for each combination
#     df['combined_name'] = df['actual_column_name'] + '_' + df['forecast_leading_days'].astype(str)

#     # Pivot table for forecast_target
#     pivot_target = df.pivot_table(index=['symbol', 'model_type', 'date'], 
#                                 columns='combined_name', 
#                                 values='forecast_target',
#                                 aggfunc='first')

#     # Renaming columns for clarity
#     pivot_target.columns = [f"forecast_target_{col}" for col in pivot_target.columns]

#     # Pivot table for forecast_leading
#     pivot_leading = df.pivot_table(index=['symbol', 'model_type', 'date'], 
#                                 columns='combined_name', 
#                                 values='forecast_leading',
#                                 aggfunc='first')

#     # Renaming columns for clarity
#     pivot_leading.columns = [f"forecast_leading_{col}" for col in pivot_leading.columns]
#     pivot_leading
#     df_merged = df.merge(pivot_target, on=['symbol', 'model_type', 'date'])


#     # Merging the pivot tables into the original DataFrame

#     df_merged = df_merged.merge(pivot_leading, on=['symbol', 'model_type', 'date'])
#     df_merged
#     # Now df_merged will have your desired structure


#     # Dropping the now redundant columns
#     columns_to_drop = ['forecast_target', 'forecast_leading', 'actual_column_name', 'forecast_leading_days', 'combined_name', 'index', 'label_date', 'label_date_reverse']
#     df_merged.drop(columns=columns_to_drop, inplace=True, errors='ignore')
#     df_merged
#     # Removing duplicate rows
#     df_final = df_merged.drop_duplicates()
#     return df_final

# pivot_forecasts(data_handler.dataframe)


In [49]:

portfolio.get_holdings()    

{'NVDA': (0.1874329846437965, 730.87),
 'ANET': (0.23752976917878899, 263.34000000000003),
 'TSCO': (0.15608163335896372, 232.6225),
 'TRV': (0.2064698835689931, 216.62),
 'BLDR': (0.3225151743353156, 185.29000000000002),
 'AZO': (0.017980830353885396, 2724.1800000000003),
 'LRCX': (0.07619844681820029, 915.7),
 'CRM': (0.16876311049746567, 287.04999999999995),
 'MAR': (0.206959971755319, 247.57),
 'META': (0.20805407521443917, 470.10005),
 'DLR': (0.26455214684255285, 140.825),
 'KLAC': (0.07000914057455329, 652.39),
 'ORLY': (0.054041219677145785, 1042.615),
 'AJG': (0.10067092063321578, 234.80345),
 'ALL': (0.3188273731227842, 159.855),
 'TDG': (0.07023722517736754, 1138.0525),
 'AMD': (0.31222853257133953, 175.81),
 'GWW': (0.07825292049395773, 943.495),
 'IBM': (0.32118735546711946, 183.57),
 'IT': (0.05598384795206437, 450.97),
 'NOW': (0.059474350891758496, 784.285),
 'AVGO': (0.04357409759172883, 1257.15895),
 'ISRG': (0.17878520284130076, 379.95500000000004),
 'FICO': (0.03305

In [50]:
import sklearn_util as sklu
import importlib
importlib.reload(sklu)

# sklu.draw_forecast_and_actuals(single_row_df)

<module 'sklearn_util' from 'c:\\src\\market\\sklearn_util.py'>

In [51]:
# import datetime
# import trader as trader
# import importlib
# importlib.reload(trader)

# data_handler = trader.DataHandler(
#     grouping_dimensions=['symbol', 'model_type'],
#     forecast_dimensions=['High', 'Low', 'Close', 'Open'],
#     date_col='Date',
#     forecast_leading_col='forecast_leading',
#     forecast_target_col='forecast_target',
#     actual_col_name='actual_column_name',
#     forecast_leading_days_col='forecast_leading_days',
#     forecast_leading_days=[1, 3, 5],
#     file_path='test/results/forecast_actuals_all_models.pkl'
#     )

# start_date = datetime.datetime.strptime('2022-01-01', '%Y-%m-%d')
# default_model_type = 'lin_regression'
# portfolio = trader.Portfolio(initial_cash=10000)
# trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
# strategy_simulator = trader.StrategySimulator(trade_simulator=trade_simulator, data_handler=data_handler, model_type=default_model_type)

# strategy_simulator.simulate_strategy(start_date)
# strategy_simulator.simulate_trades(start_date)
# print("Portfolio Cash: ", portfolio.get_cash())
# print("Portfolio Cost: ", portfolio.get_average_cost())
# print("Portfolio Value: ", portfolio.get_total_current_value())
# for symbol, holding in portfolio.holdings.items():
#     print(f"Portfolio Symbol: {symbol} Shares: {holding[0]} Price: {holding[1]} Value: {holding[1] * holding[0]}")
# print("Trading History: ")
# for order_log in trade_simulator.get_transaction_log():
#     print(f" order: {order_log[0].to_string()} time:{order_log[1]}")


In [52]:
# import datetime
# import trader as trader
# import importlib
# importlib.reload(trader)

# data_handler = trader.DataHandler(
#     grouping_dimensions=['symbol', 'model_type'],
#     forecast_dimensions=['High', 'Low', 'Close', 'Open'],
#     date_col='Date',
#     forecast_leading_col='forecast_leading',
#     forecast_target_col='forecast_target',
#     actual_col_name='actual_column_name',
#     forecast_leading_days_col='forecast_leading_days',
#     forecast_leading_days=[1, 3, 5],
#     file_path='test/results/forecast_actuals_all_models.pkl'
#     )

# start_date = datetime.datetime.strptime('2024-01-01', '%Y-%m-%d')
# default_model_type = 'lin_regression'
# portfolio = trader.Portfolio(initial_cash=10000)
# trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
# strategy_simulator = trader.StrategySimulator(trade_simulator=trade_simulator, data_handler=data_handler, model_type=default_model_type)

# strategy_simulator.simulate_strategy(start_date)
# strategy_simulator.simulate_trades(start_date)
# print("Portfolio Cash: ", portfolio.get_cash())
# print("Portfolio Cost: ", portfolio.get_average_cost())
# print("Portfolio Value: ", portfolio.get_total_current_value())
# for symbol, holding in portfolio.holdings.items():
#     print(f"Portfolio Symbol: {symbol} Shares: {holding[0]} Price: {holding[1]} Value: {holding[1] * holding[0]}")
# print("Trading History: ")
# for order_log in trade_simulator.get_transaction_log():
#     print(f" order: {order_log[0].to_string()} time:{order_log[1]}")

In [53]:
# import trader as trader
# import importlib
# importlib.reload(trader)

# initial_cash = 10000  # Starting cash for the portfolio
# portfolio = trader.Portfolio(initial_cash)
# simulator = trader.StrategySimulator(handler, portfolio)

# simulation_results = simulator.simulate_strategy('GOOG', initial_cash, 'lin_regression', price_per_trade=1)
# print(simulation_results)
# print("Portfolio Value: ", portfolio.get_value())

In [54]:
# import trader as trader
# import importlib
# importlib.reload(trader)

# initial_cash = 10000  # Starting cash for the portfolio
# portfolio = trader.Portfolio(initial_cash)
# simulator = trader.StrategySimulator(handler, portfolio)

# simulation_results = simulator.simulate_strategy('VICI', initial_cash, 'lin_regression', price_per_trade=10)
# print(simulation_results)
# print("Portfolio Value: ", portfolio.get_value())

In [55]:
# data_handler.dataframe.groupby(level=data_handler.grouping_dimensions + ['Date']).size().max()

In [56]:
# mask = data_handler.dataframe[forecast_leading_days_col].isin(forecast_leading_days)
# mask = mask & (data_handler.dataframe[actual_col_name].isin(forecast_dimensions))
# dataframe = data_handler.dataframe[mask]
# dataframe


In [57]:

# import datetime
# import pandas as pd
# import trader
# import importlib
# importlib.reload(trader)

# dataframe = pd.DataFrame({
#             'symbol': ['AAPL', 'AAPL', 'GOOG', 'GOOG'],
#             'model_type': ['type1', 'type1', 'type1', 'type1'],
#             'forecast': [100, 200, 300, 400],
#             'date': pd.to_datetime(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04']),
#             'forecast_leading': [1, 2, 3, 4],
#             'forecast_target': [100, 200, 300, 400],
#             'High': [90, 210, 290, 410],
#             'Low': [80, 125, 145, 365],
#             'Close': [85, 150, 180, 400],
#             'actual_column_name': ['High', 'High', 'Low', 'Low'],
#             'forecast_leading_days': [1, 2, 3, 4]
#         })
# data_handler = trader.DataHandler(
#     grouping_dimensions=['symbol', 'model_type'],
#     forecast_dimensions=['High', 'Low', 'Close'],
#     date_col='date',
#     forecast_leading_col='forecast_leading',
#     forecast_target_col='forecast_target',
#     actual_col_name='actual_column_name',
#     forecast_leading_days_col='forecast_leading_days',
#     forecast_leading_days=[1, 3, 5],
#     file_path=None, 
#     df=dataframe,
# )
# start_date = datetime.datetime(2021, 1, 1)
# # default_model_type = 'lin_regression'
# portfolio = trader.Portfolio(initial_cash=10000)
# trade_simulator = trader.TradeSimulator(portfolio=portfolio, data_handler=data_handler)
# strategy_simulator = trader.StrategySimulator(data_handler=data_handler, trade_simulator=trade_simulator, model_type='type1')

# date = datetime.datetime(2021, 1, 1)
# unique_symbols = ['AAPL']
# data_by_symbol = {
#     'AAPL': {
#         date: pd.DataFrame({
#         'symbol': ['AAPL'],
#         'model_type': ['type1'],
#         'date': [date],
#         'forecast_leading_High_1': [200],
#         'forecast_leading_Low_1': [150],
#         'High': [180],
#         'Low': [120],
#         'Close': [160]
#         })
#     }
# }  

# strategy_simulator.simulate_strategy_for_date(unique_symbols, data_by_symbol, date)
# df = strategy_simulator.simulate_trades(start_date)

# # data = trade_simulator.data_handler.get_stock_date_indexed(symbol='AAPL', model_type='type1')
# # data = data[data[trade_simulator.data_handler.date_col] == start_date]
# # price = data['Close'].values[0]
# # order = trader.Order('AAPL', 'buy', 10, price)
# # trade_simulator.submit_order(order, start_date)  
# # df = trade_simulator.simulate_trades(start_date, model_type='type1')


# df

In [58]:
# df['Low'].values[0]

In [59]:
# df[0]